In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

url = 'https://raw.githubusercontent.com/Alcalyxz/datasetProjectIA/main/Training-Dataset.csv'

df = pd.read_csv(url)
print(df.head(5))


   having_IP_Address  URL_Length  ...  Statistical_report  Result
0                 -1           1  ...                  -1      -1
1                  1           1  ...                   1      -1
2                  1           0  ...                  -1      -1
3                  1           0  ...                   1      -1
4                  1           0  ...                   1       1

[5 rows x 31 columns]


In [2]:
df = df.drop(['having_Sub_Domain'], axis=1)
df = df.drop(['URL_of_Anchor'], axis=1)
df = df.drop(['Links_in_tags'], axis=1)
df = df.drop(['Redirect'], axis=1)
df = df.drop(['web_traffic'], axis=1)
df = df.drop(['Links_pointing_to_page'], axis=1)

##Debido a la alta correlaciòn (0.95) con PopUpWindow se elimina una de las dos ya que le aportan
##La misma informaciòn al sistema 
df = df.drop(['Favicon'], axis=1)

In [3]:
!pip install scikit-plot

In [4]:
df = df.drop(df[df['URL_Length']==0].index)
df = df.drop(df[df['SSLfinal_State']==0].index)
df = df.drop(df[df['SFH']==0].index)

In [5]:
df.reset_index(drop=True, inplace=True)

In [75]:
##df = df.replace({-1: 0})

In [6]:
YEntrenamiento = df['Result']
XEntrenamiento = scaler.fit_transform(df.drop(['Result'],axis=1))

In [7]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
import scikitplot as skplt 


In [8]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
import scikitplot as skplt 
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE 
from sklearn.model_selection import StratifiedKFold
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [9]:
def experimentar_gbt(num_trees, X, Y):
    folds = 4
    skf = StratifiedKFold(n_splits=folds)
    resultados = pd.DataFrame()
    idx = 0
    for trees in num_trees:
        ## para almacenar los errores intermedios
        EficienciaTrain = []
        EficienciaVal = []
        for train, test in skf.split(X, Y):
            Xtrain = X[train,:]
            Ytrain = Y[train]
            Xtest = X[test,:]
            Ytest = Y[test]
            #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
            modelo= GradientBoostingClassifier(n_estimators= trees)
            modelo.fit(Xtrain, Ytrain)
            #predecir muestras de entrenamiento
            Ytrain_pred =  modelo.predict(Xtrain)
            #predecir muestras de pruebas
            Yest = modelo.predict(Xtest)
            #Evaluamos las predicciones del modelo con los datos de test
            EficienciaTrain.append(np.mean(Ytrain_pred.ravel() == Ytrain.ravel()))
            EficienciaVal.append(np.mean(Yest.ravel() == Ytest.ravel()))

        resultados.loc[idx,'número de arboles'] = trees
        resultados.loc[idx,'Accuracy de entrenamiento'] = np.mean(EficienciaTrain)
        resultados.loc[idx,'desviacion estandar entrenamiento'] = np.std(EficienciaTrain)
        resultados.loc[idx,'Accuracy de prueba'] = np.mean(EficienciaVal)
        resultados.loc[idx,'desviacion estandar prueba'] = np.std(EficienciaVal)
        idx= idx +1
        
    return (resultados)

In [10]:
arboles = [5,10,20,50,100, 150,1000]
resultados_gbt = experimentar_gbt(arboles, XEntrenamiento, YEntrenamiento)
resultados_gbt

,número de arboles,Accuracy de entrenamiento,desviacion estandar entrenamiento,Accuracy de prueba,desviacion estandar prueba
0,5.0,0.885482,0.002460,0.885482,0.007378
1,10.0,0.885482,0.002460,0.885482,0.007378
2,20.0,0.885482,0.002460,0.885482,0.007378
3,50.0,0.889988,0.001918,0.886031,0.007063
4,100.0,0.897718,0.001500,0.885041,0.008094
5,150.0,0.902077,0.001430,0.885700,0.008776
6,1000.0,0.920321,0.001839,0.881848,0.039564
